# 想法
1. 先拿nli model來train[refutes, supports, NOT ENOUGH INFO]
2. 拿一份bi_encoder生成的文件，用上面的model去predict，如果是NOT ENOUGH INFO就納入negative passage, 另外兩者就納入positive passage，再train一次bi_encoder
3. 新的結果再拿去rerank這樣

In [2]:
import torch
import tqdm
import numpy as np

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

In [3]:
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__CUDA Device Name:',torch.cuda.get_device_name(0))
print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8700
__CUDA Device Name: Tesla V100-SXM2-32GB
__CUDA Device Total Memory [GB]: 34.089730048


In [ ]:
#用colab的話不需要這句
!pip install gdown

In [ ]:
#下載訓練用資料
!gdown https://drive.google.com/drive/u/1/folders/15ebq-z8vvtM9Rl9t0OXqPkewSv9UM7oj --folder

In [4]:
#清理多餘的資料
import gc
gc.collect()

19

In [ ]:
import glob
from zipfile import ZipFile
dirname = "FACT_CHECK_COMP_DATA"
for ele in glob.glob(f"{dirname}/*.zip"):
  with ZipFile(f"{ele}") as fzp:
    fzp.extractall("./")
# del fzp
for ele in glob.glob("░V╜m╕Ω«╞╢░/*.zip"):
  with ZipFile(f"{ele}") as fzp:
    fzp.extractall("./")
del fzp
!mv ░V╜m╕Ω«╞╢░/*.jsonl ./
!mv {dirname}/*.pbz2 ./

In [5]:
import json
# to load train/test data
def get_jsonl_data(fname, dirname=""):
  return list(map(json.loads, open(f"{dirname}{fname}", "r")))

In [6]:
print_it=True

def print_fn(func):
  def wrap(*args,**kwargs):
    if print_it:
      func(*args, **kwargs)
  return wrap

@print_fn
def print_(*args, **kwargs):
  return print(*args, **kwargs)

In [7]:
import bz2 # to zip a pickle object
import pickle
def compressed_pickle(title, data):
 with bz2.BZ2File(title + '.pbz2', 'w') as f: 
  pickle.dump(data, f)
def decompress_pickle(file):
 data = bz2.BZ2File(file, 'rb')
 data = pickle.load(data)
 return data

In [8]:
flattened_wiki = decompress_pickle("cleaned_wiki.pbz2")

In [9]:
cur_title_idx = 0 # need to accumulate to generate starting index
lookup_title_idx = []
passages = []
for data in flattened_wiki:
    lookup_title_idx.append(cur_title_idx)
    # rewrite the 'ele.index('\t')' part ref to full_pipeline.ipynb
    _psg = []
    line_length = 0
    for ele in data['lines'].split('\n'):
      startidx = ele.index('\t')
      if "。" in ele:
        endidx = ele.index("。")
      else:
        endidx = len(ele)
      txt = ele[startidx:endidx].replace('\t','').replace('。','').replace(' ','') # not sure whether to comment . out
      if len(txt) > 0:
        # to keep search result accurate, we put 'replace('\t')' here
        # instead of at len(txt) > 0
        # -> len(passages[idx]) == 0 will occur
        #_psg.append(txt.replace('\t',''))
        _psg.append(txt)
        line_length += 1
    if line_length == 0:
      continue
    passages += _psg # append passages
    cur_title_idx += line_length # increase start_idx by number of lines of the article

# check length
len(lookup_title_idx), lookup_title_idx[:5], len(passages)

(1181684, [0, 5, 12, 15, 19], 4319174)

In [10]:
## faster implementation (with built-in package)
from bisect import bisect_right
def find_le(a, x, index=True):
    """
    a: array to search
    x: element of interest, might not belong to the list
    index: return the index if True, otherwise, a[idx]
    'Find rightmost value <= x'
    """
    i = bisect_right(a, x)
    if i:
        if index:
            return i-1
        return a[i-1]
    raise ValueError

In [11]:
def lookup_title(h, top_k=256, print_it = True, wiki=flattened_wiki):
  "returns up to k titles & line number of single search result, default top_k=5"
  ret = []
  for idx in range(top_k):
    crit = h[idx]['corpus_id']
    title_idx = find_le(lookup_title_idx, crit)
    line_no = crit - lookup_title_idx[title_idx]
    if print_it:
      print("(#%d)" % (idx+1))
      print('Original Title:', wiki[title_idx]['id'])
      print('\nThe selected passage (see if line_no is right):\n%s\n' % wiki[title_idx]['lines'].split('\n')[line_no])
    _r = [wiki[title_idx]['id'], line_no]
    ret.append(_r)
  return ret

In [12]:
# SBERT sentence transformer class
!pip install -U sentence-transformers -q

In [13]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util, datasets, models, losses

In [14]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

In [15]:
title_to_idx = {ele['id']: idx for idx, ele in enumerate(flattened_wiki)}

In [16]:
def parse_evidence(evidence):
  # evidence: list of (int, int, str, int)
  to_refer = [] # list of (title, line_no)
  for evd in evidence:
    to_refer.append(evd[-2:])
  # do the lookup (to wiki)
  ret = []
  for ele in to_refer:
    title, line_no = ele
    # do error handling: if title in title_to_idx (aka, valid data) -> everything works
    # else: skip it
    if title not in title_to_idx:
      print("fuck it")
      continue
    t_idx = title_to_idx[title] # use a str:int map to lookup title index in 'flattened_wiki'
    # remove numbers up front of each sentence
    _text = flattened_wiki[t_idx]['lines'].split('\n')[line_no] 
    _idx = _text.index('\t')
    if "。" in _text:
      end_idx = _text.index('。')
    else:
      end_idx = len(_text)
    # the actual sentence
    text_ = _text[_idx:end_idx].replace('\t','').replace(' ','')
    ret.append(text_)
  # concat all the evidence text to 1 sentence and return
  return " ".join(ret)
res = [] # result of the parsed evidence: (query, related)

In [17]:
res = []
resans = []

In [18]:
def label2int(a):
  if a=="refutes":
    return 0
  if a=="supports":
    return 2
  if a=="NOT ENOUGH INFO":
    return 1

In [ ]:
#baseline code part 1+2 results
!gdown --folder https://drive.google.com/drive/u/1/folders/1-HQC9UzEYcyM5VbSxBJqONawmVW6D20l?fbclid=IwAR38oIhEC5UzFet_d02YNYCHGdd26Ixl4yNDFV1vnymae1og7IppLBN_GA8

In [20]:
torch.cuda.empty_cache()

In [22]:
top_k = 256 #change if needed

In [23]:
import pandas as pd

with open('prediction_results/train_doc64sent64_with_0522.jsonl',encoding='utf8') as predict_source:
    Total_Predict = pd.read_json(predict_source, lines=True)
with open('prediction_results/dev_doc64sent64_with_0522.jsonl',encoding='utf8') as predictval_source:
    PredictVal_df = pd.read_json(predictval_source, lines=True)

In [24]:
import pandas as pd

with open('public_test_data.jsonl',encoding='utf8') as ts1:
    tdf1 = pd.read_json(ts1, lines=True)
with open('private_test_data.jsonl',encoding='utf8') as ts2:
    tdf2 = pd.read_json(ts2, lines=True)

total_test = [tdf1, tdf2]
Total_Test = pd.concat(total_test, ignore_index=True)

In [26]:
import random

In [ ]:
#for NLI Dataset
theclaims = []
theevidences= []
thelabels = []
for i in range(len(Total_Predict['label'])):
    if Total_Predict['label'][i]=="NOT ENOUGH INFO":
        group = []
        if Total_Predict['predicted_evidence'][i]==[]:
            continue
        for singleevi in Total_Predict['predicted_evidence'][i]:
            if Total_Predict['id'][i]!=Total_Predict['id'][Total_Predict['id']==Total_Predict['id'][i]].tolist()[0]:
                print("fuckers")
            title = singleevi[0]
            lineindex = singleevi[1]
            original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
            startidx = original.index('\t')
            if "。" in original:
                endidx = original.index("。")
            else:
                endidx = len(original)
            group.append(original[startidx:endidx].replace('\t','').replace(' ',''))
            thelabels.append(1)
            theevidences.append(original[startidx:endidx].replace('\t','').replace(' ',''))
            theclaims.append(Total_Predict['claim'][i])
        for uu in range(12):
            evistr = ""
            evilist = random.choices(group, k = 5)
            random.shuffle(evilist)
            for tt in range(5):
                evistr+=evilist[tt]
            thelabels.append(1)
            theevidences.append(evistr)
            theclaims.append(Total_Predict['claim'][i])
    elif Total_Predict['label'][i]=="refutes":
        #表示是反對的
        group = []
        for mm in range(len(Total_Predict['evidence'][i])):
            for singleevi in Total_Predict['evidence'][i][mm]:
                #print(singleevi)
                title = singleevi[2]
                lineindex = singleevi[3]
                if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
                startidx = original.index('\t')
                if "。" in original:
                    endidx = original.index("。")
                else:
                    endidx = len(original)
                evidenceforgroup = original[startidx:endidx].replace('\t','').replace(' ','')
                group.append(evidenceforgroup)
        if Total_Predict['predicted_evidence'][i]==[]:
            continue
        neutralgroup = []
        for neutralevi in Total_Predict['predicted_evidence'][i]:
            #print(neutralevi)
            title = neutralevi[0]
            lineindex = neutralevi[1]
            original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
            startidx = original.index('\t')
            if "。" in original:
                endidx = original.index("。")
            else:
                endidx = len(original)
            singleneutral = original[startidx:endidx].replace('\t','').replace(' ','')
            if not singleneutral in group:
                neutralgroup.append(singleneutral)
        group.append(Total_Predict['claim'][i])
        for j in range(len(group)):
            if len(neutralgroup)!=0:
                for pp in range(12):
                    evistr1 = ""
                    evilist1 = random.choices(neutralgroup, k = 5)
                    random.shuffle(evilist1)
                    for tt in range(5):
                        evistr1+=evilist1[tt]
                    theclaims.append(group[j])
                    thelabels.append(1)
                    theevidences.append(evistr1)
                fcount = 0
                for ff in range(len(neutralgroup)):
                    if fcount<12:
                        theclaims.append(group[j])
                        thelabels.append(1)
                        theevidences.append(neutralgroup[ff])
                        fcount+=1
            for k in range(len(group)):
                if j!=k:
                    if len(neutralgroup)!=0:
                        for uu in range(12):
                            evistr = ""
                            evilist = random.choices(neutralgroup, k = 4)
                            evilist.append(group[k])
                            random.shuffle(evilist)
                            for tt in range(5):
                                evistr+=evilist[tt]
                            theclaims.append(group[j])
                            thelabels.append(0)
                            theevidences.append(evistr)
                    theclaims.append(group[j])
                    theevidences.append(group[k])
                    thelabels.append(0)
    elif Total_Predict['label'][i]=="supports":
        #表示是反對的
        group = []
        for mm in range(len(Total_Predict['evidence'][i])):
            for singleevi in Total_Predict['evidence'][i][mm]:
                #print(singleevi)
                title = singleevi[2]
                lineindex = singleevi[3]
                if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
                startidx = original.index('\t')
                if "。" in original:
                    endidx = original.index("。")
                else:
                    endidx = len(original)
                evidenceforgroup = original[startidx:endidx].replace('\t','').replace(' ','')
                group.append(evidenceforgroup)
        if Total_Predict['predicted_evidence'][i]==[]:
            continue
        neutralgroup = []
        for neutralevi in Total_Predict['predicted_evidence'][i]:
            #print(neutralevi)
            title = neutralevi[0]
            lineindex = neutralevi[1]
            original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
            startidx = original.index('\t')
            if "。" in original:
                endidx = original.index("。")
            else:
                endidx = len(original)
            singleneutral = original[startidx:endidx].replace('\t','').replace(' ','')
            if not singleneutral in group:
                neutralgroup.append(singleneutral)
        group.append(Total_Predict['claim'][i])
        for j in range(len(group)):
            if len(neutralgroup)!=0:
                for pp in range(12):
                    evistr1 = ""
                    evilist1 = random.choices(neutralgroup, k = 5)
                    random.shuffle(evilist1)
                    for tt in range(5):
                        evistr1+=evilist1[tt]
                    theclaims.append(group[j])
                    thelabels.append(1)
                    theevidences.append(evistr1)
                fcount = 0
                for ff in range(len(neutralgroup)):
                    if fcount<12:
                        theclaims.append(group[j])
                        thelabels.append(1)
                        theevidences.append(neutralgroup[ff])
                        fcount+=1
            for k in range(len(group)):
                if j!=k:
                    if len(neutralgroup)!=0:
                        for uu in range(12):
                            evistr = ""
                            evilist = random.choices(neutralgroup, k = 4)
                            evilist.append(group[k])
                            random.shuffle(evilist)
                            for tt in range(5):
                                evistr+=evilist[tt]
                            theclaims.append(group[j])
                            thelabels.append(2)
                            theevidences.append(evistr)
                    theclaims.append(group[j])
                    theevidences.append(group[k])
                    thelabels.append(2)

In [ ]:
#for bi_encoder
#這個不用跑！(因為我們最後的結構並沒有使用Bi-Encoder)
"""theclaims = []
thepositives= []
thenegatives = []
for i in range(len(Total_Predict['label'])):
    if Total_Predict['label'][i]=="NOT ENOUGH INFO":
        continue
    else:
        #表示是有關的
        group = []
        for mm in range(len(Total_Predict['evidence'][i])):
            for singleevi in Total_Predict['evidence'][i][mm]:
                #print(singleevi)
                title = singleevi[2]
                lineindex = singleevi[3]
                if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
                startidx = original.index('\t')
                if "。" in original:
                    endidx = original.index("。")
                else:
                    endidx = len(original)
                evidenceforgroup = original[startidx:endidx].replace('\t','').replace(' ','')
                group.append(evidenceforgroup)
        if Total_Predict['predicted_evidence'][i]==[]:
            continue
        neutralgroup = []
        for neutralevi in Total_Predict['predicted_evidence'][i]:
            #print(neutralevi)
            title = neutralevi[0]
            lineindex = neutralevi[1]
            original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
            startidx = original.index('\t')
            if "。" in original:
                endidx = original.index("。")
            else:
                endidx = len(original)
            singleneutral = original[startidx:endidx].replace('\t','').replace(' ','')
            if not singleneutral in group:
                neutralgroup.append(singleneutral)
        group.append(Total_Predict['claim'][i])
        for j in range(len(group)):
            for k in range(len(group)):
                if j!=k:
                    neutralcounter = 0
                    for l in range(len(neutralgroup)):
                        if neutralcounter<5:
                            theclaims.append(group[j])
                            thepositives.append(group[k])
                            thenegatives.append(random.choice(neutralgroup))
                            neutralcounter+=1"""

In [ ]:
#確認三個數量有一樣
print(len(theclaims))
print(len(thepositives))
print(len(thenegatives))

In [ ]:
forcalc = np.array(thelabels)

In [ ]:
TotalLength = len(theclaims)
class_weights = [TotalLength/3/np.sum(forcalc==0),TotalLength/3/np.sum(forcalc==1),TotalLength/3/np.sum(forcalc==2)]

In [ ]:
example_weights = [class_weights[e] for e in thelabels]

In [ ]:
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
Sampler = WeightedRandomSampler(example_weights,len(thelabels))
cross_train_data = [InputExample(texts=[theclaims[i],theevidences[i]],label = thelabels[i]) for i in range(len(thelabels))]
cross_loader = DataLoader(cross_train_data, sampler=Sampler, batch_size=16)

In [27]:
#Classifier
Cross_Encoder = CrossEncoder('cross-encoder/nli-deberta-v3-base', num_labels = 3)

/home/twssbvh778/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
#這是我們train出來結果最好的weight
!gdown https://drive.google.com/u/4/uc?id=1-eOGzJj5wDhC2qDv8XIoCo79AL3-QJ94&export=download

In [28]:
#如果有下載上面的連結的話，再跑這句來load weights進去；否則不需要，直接開始train即可
Cross_Encoder.model.load_state_dict(torch.load('PleaseBeUseful2/pytorch_model.bin'))

<All keys matched successfully>

In [54]:
torch.cuda.empty_cache()

In [31]:
gc.collect()

24

In [ ]:
Cross_Encoder.fit(train_dataloader=cross_loader, epochs=1, use_amp = True, show_progress_bar=True)

In [ ]:
#這邊可以更換不同的final label prediction method，詳見文件敘述
truepredict = 0
totalpredict = 0
y_pred = []
y_true = []
for i in tqdm(range(len(PredictVal_df['predicted_evidence']))):
    evicounter2 = 0
    votes = [0,0,0]
    for singleevi in PredictVal_df['predicted_evidence'][i]:
        #print(singleevi)
        if evicounter2<5:
            title = singleevi[0]
            lineindex = singleevi[1]
            if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                continue
            original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
            startidx = original.index('\t')
            if "。" in original:
                endidx = original.index("。")
            else:
                endidx = len(original)
            finalevi = Cross_Encoder.predict([PredictVal_df['claim'][i],original[startidx:endidx].replace('\t','').replace(' ','')])
            votes[np.argmax(finalevi)]+=1
            evicounter2+=1
    if votes[0]>0:
        finalans=0
        y_pred.append(0)
    elif votes[2]>0:
        finalans = 2
        y_pred.append(2)
    else:
        finalans = 1
        y_pred.append(1)
    y_true.append(label2int(PredictVal_df['label'][i]))
    if finalans==label2int(PredictVal_df['label'][i]):
        truepredict+=1
    totalpredict+=1
    #print(totalpredict)
print((float)(truepredict/totalpredict))

In [ ]:
#用confusion matrix看一下結果
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import seaborn as sn

actual = y_true
predicted = y_pred

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0,1,2])

#sn.heatmap(confusion_matrix, annot=True,cmap="OrRd")
cm_display.plot()
plt.show()

In [150]:
#如果有需要儲存model weights結果的話，自行修改檔名即可
import shutil
shutil.make_archive("BBE_Model", 'zip', "best_bi_encoder")

In [ ]:
#其他的evidence prediction results，如果想要測試的話可以隨意更換
!gdown --folder https://drive.google.com/drive/u/1/folders/1-HQC9UzEYcyM5VbSxBJqONawmVW6D20l?fbclid=IwAR38oIhEC5UzFet_d02YNYCHGdd26Ixl4yNDFV1vnymae1og7IppLBN_GA8

In [57]:
with open('test_doc64sent64_now.jsonl',encoding='utf8') as test_source:
    Test_df = pd.read_json(test_source, lines=True)

In [56]:
torch.cuda.empty_cache()

In [58]:
#submission template
with open("public_private_submission_template.jsonl", encoding = 'utf8') as thetemplate:
    subtemplate = pd.read_json(thetemplate, lines = True)

In [59]:
subtemplate

,id,predicted_label,predicted_evidence
0,63,not enough info,NaN
1,74,not enough info,NaN
2,77,not enough info,NaN
3,78,not enough info,NaN
4,82,not enough info,NaN
...,...,...,...
9033,22111,not enough info,NaN
9034,22112,not enough info,NaN
9035,22113,not enough info,NaN
9036,22114,not enough info,NaN


In [60]:
#將前五高分的evidence列為final evidence
final_evidence = []
for i in range(len(subtemplate['id'])):
    final_evidence.append(Test_df['predicted_evidence'][Test_df['id']==subtemplate['id'][i]].tolist()[0][:5])

[[['女真', 4], ['漢姓', 0], ['漢服', 0], ['女真', 1], ['女真', 0]],
 [['昭和天皇', 16], ['昭和天皇', 0], ['昭和天皇', 4], ['昭和', 0], ['天皇', 1]],
 [['昭和天皇', 0], ['昭和天皇', 16], ['昭和', 0], ['昭和天皇', 4], ['天皇', 1]],
 [['昭和天皇', 16], ['昭和天皇', 0], ['昭和天皇', 4], ['1989', 5], ['昭和天皇', 17]],
 [['象', 0], ['大象_(年號)', 0], ['象', 5], ['唐朝', 3], ['象', 1]],
 [['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 6], ['樂山大佛', 6]],
 [['臺北', 1], ['臺北', 0], ['臺北', 4], ['臺北', 5]],
 [['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 6], ['樂山大佛', 6]],
 [['非營利組織', 10], ['非營利組織', 7], ['非營利組織', 14], ['非營利組織', 0], ['非營利組織', 3]],
 [['非營利組織', 0], ['非營利組織', 11], ['組織', 4], ['非營利組織', 14], ['非營利組織', 3]],
 [['非營利組織', 0], ['非營利組織', 14], ['非營利組織', 10], ['組織', 4], ['組織', 2]],
 [['非營利組織', 0], ['目標', 1], ['非營利組織', 11], ['非營利組織', 14], ['非營利組織', 10]],
 [['非營利組織', 0], ['非營利組織', 14], ['非營利組織', 10], ['非營利組織', 7], ['組織', 2]],
 [['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 1], ['樂山大佛', 6], ['樂山大佛', 6]],
 [['樂山大佛', 1], ['樂山大佛', 6], ['樂山大佛', 0], ['樂山大佛', 2], ['樂山大佛', 5]],
 [

In [61]:
subtemplate['predicted_evidence'] = final_evidence

In [62]:
subtemplate

,id,predicted_label,predicted_evidence
0,63,not enough info,"[[女真, 4], [漢姓, 0], [漢服, 0], [女真, 1], [女真, 0]]"
1,74,not enough info,"[[昭和天皇, 16], [昭和天皇, 0], [昭和天皇, 4], [昭和, 0], [天..."
2,77,not enough info,"[[昭和天皇, 0], [昭和天皇, 16], [昭和, 0], [昭和天皇, 4], [天..."
3,78,not enough info,"[[昭和天皇, 16], [昭和天皇, 0], [昭和天皇, 4], [1989, 5], ..."
4,82,not enough info,"[[象, 0], [大象_(年號), 0], [象, 5], [唐朝, 3], [象, 1]]"
...,...,...,...
9033,22111,not enough info,"[[張惠妹, 0], [張惠妹, 3], [天后, 3], [張惠妹, 8], [張惠妹, 9]]"
9034,22112,not enough info,"[[張惠妹, 13], [張惠妹, 0], [張惠妹, 14], [張惠妹, 3], [張惠..."
9035,22113,not enough info,"[[張惠妹, 0], [張惠妹, 13], [張惠妹, 14], [張惠妹, 3], [張惠..."
9036,22114,not enough info,"[[張惠妹, 0], [張惠妹, 13], [張惠妹, 14], [張惠妹, 3], [張惠..."


In [63]:
#Classification
finallabels = []
fuckidx = 0
for i in range(len(Test_df)):
    #votes = [0,0,0]
    currentsentence=""
    for singleevidence in subtemplate['predicted_evidence'][i]:
        #print(singleevidence)
        original = flattened_wiki[title_to_idx[singleevidence[0]]]['lines'].split('\n')[singleevidence[1]]
        start_idx = original.index('\t')
        if "。" in original:
            end_idx = original.index("。")
        else:
            end_idx = len(original)
        sentence = original[start_idx:end_idx].replace('\t','').replace(' ','')
        currentsentence+=sentence
    originalans = Cross_Encoder.predict([Test_df['claim'][Test_df['id']==subtemplate['id'][i]].tolist()[0],currentsentence])
    finalans = np.argmax(originalans)
    if finalans==0:
        finallabels.append("refutes")
    if finalans==1:
        finallabels.append("NOT ENOUGH INFO")
    if finalans==2:
        finallabels.append("supports")

In [64]:
subtemplate['predicted_label'] = finallabels

In [65]:
#一些answer的debug，避免系統算分錯誤
for i in range(len(subtemplate['predicted_label'])):
    if subtemplate['predicted_label'][i]=="NOT ENOUGH INFO":
        subtemplate['predicted_evidence'][i] = "None"

/tmp/ipykernel_7045/813413962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subtemplate['predicted_evidence'][i] = "None"


In [66]:
#一些answer的debug，避免系統算分錯誤
for i in range(len(subtemplate['predicted_label'])):
    if subtemplate['predicted_evidence'][i]==[]:
        subtemplate['predicted_label'][i] = "NOT ENOUGH INFO"

In [67]:
#檔名可以自行更改
subtemplate.to_json("submission_pbu2_bbe_combine_finaltry2.jsonl", orient="records",lines=True,force_ascii=False)

In [34]:
#這個不需要
"""def search_rewrite(query, passages=passages, top_k = 256):
    # query: list of string
    #print_("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    # encode() will handle the case when query: str instead of list
    question_embedding = Bi_Encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=256)

    ret = {} # dict; query: topk result
    assert len(hits) == len(query)
    for i in range(len(hits)):
      # hope there's no restriction on length of key in dict()
      ret[query[i]] = _search(hits[i], query[i], passages)
    
    return ret



def _search(hits, query, passages):
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    #cross_scores = Reranker.predict(cross_inp)

    # Sort results by the cross-encoder scores
    #for idx in range(len(cross_scores)):
    #    hits[idx]['cross-score'] = cross_scores[idx]
    # print_ query
    #print_("\nInput question:", query)
    # Output of top-5 hits from bi-encoder
    #print_("\n-------------------------\n")
    #print_("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    #for hit in hits[0:3]:
    #    print_("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    #print_("\n-------------------------\n")
    #print_("Top-3 Cross-Encoder Re-ranker hits")
    #hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    #for hit in hits[0:3]:
    #    print_("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))
    return hits # for title lookup"""